In [26]:
import pandas as pd
import numpy as np
import config
import os
from datetime import date
from datetime import datetime

file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\38_x5_report_parser'
file_name = 'test.xlsx'

df = pd.read_excel(os.path.join(file_path, file_name))

In [27]:
df

,date,category,amount
0,2025-01-01,olv,0
1,2025-01-01,banner,200
2,2025-01-02,olv,110
3,2025-01-02,banner,200
4,2025-01-03,olv,150
5,2025-01-04,banner,220
6,2025-01-05,olv,0
7,2025-01-06,olv,0
8,2025-01-07,olv,0
9,2025-02-10,olv,330


In [28]:
df['date'] = pd.to_datetime(df['date'])

In [29]:
df = df.sort_values(['date', 'category'])
df = df.reset_index(drop='True')

In [30]:
df['first_not_blank'] = df[df['amount'] > 0].groupby('category')['date'].transform('min')
df['first_not_blank'] = df['first_not_blank'].fillna('0')

In [31]:
df

,date,category,amount,first_not_blank
0,2025-01-01,banner,200,2025-01-01 00:00:00
1,2025-01-01,olv,0,0
2,2025-01-02,banner,200,2025-01-01 00:00:00
3,2025-01-02,olv,110,2025-01-02 00:00:00
4,2025-01-03,olv,150,2025-01-02 00:00:00
5,2025-01-04,banner,220,2025-01-01 00:00:00
6,2025-01-05,olv,0,0
7,2025-01-06,olv,0,0
8,2025-01-07,olv,0,0
9,2025-01-08,banner,250,2025-01-01 00:00:00


In [6]:
# df['amount_prev'] = np.where((df['first_not_blank']!=0)&(df['date']==df['first_not_blank']), 
#                            df['amount'], df.groupby('category')['amount'].shift(1))

In [7]:
# df['amount_prev'] = df['amount_prev'].fillna(0)

In [32]:
df['amount_not_blank'] = df.groupby(['category'])['amount'].cummax()

In [33]:
df

,date,category,amount,first_not_blank,amount_not_blank
0,2025-01-01,banner,200,2025-01-01 00:00:00,200
1,2025-01-01,olv,0,0,0
2,2025-01-02,banner,200,2025-01-01 00:00:00,200
3,2025-01-02,olv,110,2025-01-02 00:00:00,110
4,2025-01-03,olv,150,2025-01-02 00:00:00,150
5,2025-01-04,banner,220,2025-01-01 00:00:00,220
6,2025-01-05,olv,0,0,150
7,2025-01-06,olv,0,0,150
8,2025-01-07,olv,0,0,150
9,2025-01-08,banner,250,2025-01-01 00:00:00,250


In [34]:
df['increment'] = np.where((df['first_not_blank']!=0)&(df['date']==df['first_not_blank']), 
                           df['amount_not_blank'], df['amount_not_blank'] - df.groupby('category')['amount_not_blank'].shift(1))

In [35]:
df['increment'] = df['increment'].fillna(0)

In [36]:
df

,date,category,amount,first_not_blank,amount_not_blank,increment
0,2025-01-01,banner,200,2025-01-01 00:00:00,200,200.0
1,2025-01-01,olv,0,0,0,0.0
2,2025-01-02,banner,200,2025-01-01 00:00:00,200,0.0
3,2025-01-02,olv,110,2025-01-02 00:00:00,110,110.0
4,2025-01-03,olv,150,2025-01-02 00:00:00,150,40.0
5,2025-01-04,banner,220,2025-01-01 00:00:00,220,20.0
6,2025-01-05,olv,0,0,150,0.0
7,2025-01-06,olv,0,0,150,0.0
8,2025-01-07,olv,0,0,150,0.0
9,2025-01-08,banner,250,2025-01-01 00:00:00,250,30.0


In [ ]:
# df['test'] = df.query('amount > 0').groupby('category')['date'].transform('min')

In [ ]:
# df['date'] = pd.to_datetime(df['date'])
# df['start_date'] = df.groupby('category')['date'].transform('min')

In [24]:
df

,date,category,amount,first_not_blank,amount_not_blank,increment
0,2025-01-01,banner,200,2025-01-01 00:00:00,200,200.0
1,2025-01-01,olv,0,0,0,0.0
2,2025-01-02,banner,200,2025-01-01 00:00:00,200,0.0
3,2025-01-02,olv,110,2025-01-02 00:00:00,110,110.0
4,2025-01-03,olv,150,2025-01-02 00:00:00,150,40.0
5,2025-01-04,banner,220,2025-01-01 00:00:00,220,20.0
6,2025-01-05,olv,0,0,150,0.0
7,2025-01-06,olv,0,0,150,0.0
8,2025-01-07,olv,0,0,150,0.0
9,2025-01-08,banner,250,2025-01-01 00:00:00,250,30.0


In [25]:
df.groupby('category')['increment'].sum()

category
banner    250.0
olv       330.0
Name: increment, dtype: float64